In [1]:
import pandas as pd
# Import all the tools we need
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('ReportOnClutter.csv')

In [3]:
df.isna().sum()

Target                   0
Duration Of Intercept    0
TOLA                     0
Max Hit Count            0
Frequency(MHz)           0
Pulse Width(us)          0
arr_fPRI[1]              0
Bandwidth(MHz)           0
Est_LF_Lat(deg)          0
Est_LF_Long(deg)         0
MaxAmplitude(dBm)        0
dtype: int64

In [4]:
df = df.drop('TOLA', axis=1)

In [5]:
df_1 = df[df['Target'] == 1]
df_0 = df[df['Target'] == 0]
df_0 = df_0.sort_values(by='Frequency(MHz)', ascending=False)
df_1 = df_1.sort_values(by='Frequency(MHz)', ascending=False)

In [6]:
# One-Class SVM
model_svm_01 = OneClassSVM(kernel='poly', gamma=0.5, nu=0.9)

X = df_1.drop('Target', axis=1)
y = df_1['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model_svm_01.fit(X_train, y_train)

# Predict on the test data
y_pred_svm = model_svm_01.predict(X_test)

# Convert the predictions to 1 (anomaly) and 0 (normal)
y_pred_svm = np.where(y_pred_svm == -1, 1, 0)

# Calculate and print the accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
#print(f'Prediction:', y_pred_svm)
print(f"Accuracy of One-Class SVM: {accuracy_svm}")


Accuracy of One-Class SVM: 0.83984375


Automation of task

Given freq ranges determine the 8 parameters and feed into the csv files for direct analysis.The params are :
1. Target	
2. Duration Of Intercept	
3. TOLA	
4. Max Hit Count	
5. Min Frequency(MHz)
6. Max 	Frequency(MHz)
6. Max Pulse Width(us)	
7. Max arr_fPRI[1]	
8. Max Bandwidth(MHz)	
9. Est_LF_Lat(deg)	
10. Est_LF_Long(deg)	
11. Max Amplitude(dBm)
12. Min Amplitude(dBm)
 

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Prepare the data
pursuit = pd.concat(map(pd.read_csv, ["D:\\ELSEC\\SortiModel\\data\\1.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\2.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\3.csv",
                                      "D:\\ELSEC\\SortiModel\\data\\4.csv"]), ignore_index=True)

# Data preprocessing steps
# Ensure 'Target' column exists and is the last column in the dataframe
pursuit['Target'] = pursuit.apply(lambda row: 0 if row['WeaponSystem'] == 'T5' else 1, axis=1)
pursuit = pursuit.drop(['Emitter Name'], axis=1)  # Drop 'Emitter Name' column
pursuit = pd.get_dummies(pursuit, drop_first=True)  # One-hot encode categorical variables
X = pursuit.drop('Target', axis=1)
y = pursuit['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the data to numpy arrays
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Build the model
model = Sequential([
    Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

# Predict on the test set
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.where(y_pred > 0.5, 1, 0)

# Calculate accuracy
accuracy = np.mean(y_pred_classes == y_test)
print(f'Prediction Accuracy: {accuracy:.4f}')


Epoch 1/100


D:\ELSEC\SortiModel\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


493/493 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8472 - loss: 0.3245 - val_accuracy: 1.0000 - val_loss: 4.6736e-05
Epoch 2/100
493/493 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9987 - loss: 0.0079 - val_accuracy: 0.9992 - val_loss: 0.0024
Epoch 3/100
493/493 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9988 - loss: 0.0054 - val_accuracy: 0.9997 - val_loss: 0.0015
Epoch 4/100
493/493 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9993 - loss: 0.0029 - val_accuracy: 0.9997 - val_loss: 0.0015
Epoch 5/100
493/493 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9998 - loss: 4.8024e-04 - val_accuracy: 0.9990 - val_loss: 0.0061
Epoch 6/100
493/493 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9996 - loss: 0.0012 - val_accuracy: 0.9997 - val_loss: 4.9430e-04
Epoch 7/100
493/493 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9999 - loss: 0.0012 - val_accuracy: 0.9992 - val_loss: 0.0018
Epoch 8/100
493/493 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9999 - loss: 0.0026 - val_accu

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = Dense(dim, activation="relu")(x)
        x = Dropout(mlp_dropout)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    return Model(inputs, outputs)

input_shape = (X_train.shape[1], 1)  # assuming your data is reshaped to (num_samples, num_features, 1)
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=256,
    num_transformer_blocks=4,
    mlp_units=[128, 64],
    dropout=0.1,
    mlp_dropout=0.1,
)

model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)

model.summary()

# Assuming X_train, y_train, X_test, y_test are already defined and X_train/X_test reshaped to (num_samples, num_features, 1)
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32
)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Accuracy of Transformer model: {accuracy}")


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 270, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 270, 1)    │          2 │ input_layer_1[0]… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 270, 1)    │      7,169 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 270, 1)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 270, 1)    │          0 │ dropout_3[0][0],  │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 270, 1)    │          2 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 270, 256)  │        512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 270, 256)  │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 270, 1)    │        257 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 270, 1)    │          0 │ dense_5[0][0],    │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 270, 1)    │          2 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 270, 1)    │      7,169 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 270, 1)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 270, 1)    │          0 │ dropout_6[0][0],  │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 270, 1)    │          2 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 270, 256)  │        512 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 270, 256)  │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 270, 1)    │        257 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 270, 1)    │          0 │ dense_7[0][0],    │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 74,777 (292.10 KB)

 Trainable params: 74,777 (292.10 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
 25/493 ━━━━━━━━━━━━━━━━━━━━ 12:42 2s/step - accuracy: 0.5460 - loss: 243.2033

KeyboardInterrupt: 